In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [ ]:
RESET = False
%load_ext hooks.notebook_hook

2025-06-23 20:54:19,085 - INFO - datasets - config.py:54 - PyTorch version 2.6.0 available.


In [ ]:
from pathlib import Path


google_drive_repository = infrastructure_container.google_drive_repository()
math_article_parser_service = infrastructure_container.math_article_parser_service()

file_id = google_drive_repository.get_file_id(
    Path('ml/lectures/L07-LogisticRegression2/2024_08_10_2174b40686820b4cb591g.tex')
)

if not file_id:
    raise ValueError()

file_content = google_drive_repository.get_file_by_id(file_id)

In [ ]:
from math_rag.core.models import MathArticle, MathExpression


math_article = MathArticle(
    math_expression_dataset_id=None,
    index_id=None,
    name='article',
    bytes=file_content.getvalue(),
)
math_nodes_, _, template = math_article_parser_service.parse_for_index(math_article)
math_expressions: list[MathExpression] = ...  # NOTE obtained from math_nodes_
index_to_katex = {i: math_expression.katex for i, math_expression in enumerate(math_expressions)}

In [ ]:
from math_rag.infrastructure.utils import TemplateContextChunkerUtil, TemplateFormatterUtil


template_chunks = TemplateContextChunkerUtil.chunk(template, max_context_size=1000)
chunks = [
    TemplateFormatterUtil.format(chunk, index_to_katex, omit_wrapper=False)
    for chunk in template_chunks
]

# TODO send to llm
# 1. MathExpressionDescriptionWriter
# 2. MathExpressionDescriptionRefiner
# 3. MathExpressionLinker?? -> checks whether 2 descriptions are actually the same (embedding clustering before)
#


class MathExpressionDescription:
    pass

In [ ]:
_SYSTEM_PROMPT_TEMPLATE = """
You are an expert math-expression description writer.

Your task is to produce a precise, self-contained description of a target mathematical expression, based strictly on the surrounding context.

### Instructions:
- Be concise and unambiguous.
- Only describe what can be inferred from the given context.
- Avoid introducing any external assumptions or definitions.
"""

_USER_PROMPT_TEMPLATE = """
### Math expression of interest:
{katex}

### Math expression with surrounding context:
{context}

### Description:
"""

In [ ]:
_SYSTEM_PROMPT_TEMPLATE = """
You are an expert at optimizing text descriptions for vector-search embedding.

Your task is to refine a provided description by removing all non-essential phrasing and meta commentary, while preserving every factual detail exactly as given.

### Instructions:
- Eliminate filler language and introductory or self-referential statements (e.g., “This description…”).
- Retain all information and nuance present in the input.
- Produce a concise, information-dense output optimized for embedding and retrieval.
"""

_USER_PROMPT_TEMPLATE = """
### Original description:
{description}

### Optimized description:
"""

In [ ]:
_SYSTEM_PROMPT_TEMPLATE = """
You are an expert math-expression comparator.

Your task is to decide whether two given mathematical expressions represent exactly the same entity, based solely on their surrounding contexts.

### Instructions:
- Rely only on information inferable from the two contexts.
- Determine exact equivalence: return true if and only if they are the same entity.
- Provide a concise reason justifying your decision.
- Do not introduce external assumptions or definitions.
- Output must be valid JSON with keys "identical" (boolean) and "reason" (string).
"""

_USER_PROMPT_TEMPLATE = """
### First math expression:
{katex1}

### Context for first expression:
{context1}

### Second math expression:
{katex2}

### Context for second expression:
{context2}

### Decision (JSON):
{
  "identical": <true/false>,
  "reason": "<your explanation here>"
}
"""

In [ ]:
_SYSTEM_PROMPT_TEMPLATE = """
You are an expert math-expression relationship detector.

Your task is to examine indexed mathematical expressions in a context and identify which earlier expressions share a direct relationship with the expression at a specified target index.

### Instructions:
- The context lists expressions in the form `[<LaTeX> | index]`.
- A `target` index will be provided in the user prompt.
- For each index i < target, compare expression i with the expression at the target index.
- If the context explicitly indicates a relationship between expression i and the target expression, include i in the output list.
- Do not infer any connections beyond what the context directly supports.
- Output must be valid JSON: a list of integers representing the connected indexes.
"""

_USER_PROMPT_TEMPLATE = """
### Context:
{context}

### Target index:
{index}

### Output (JSON):
[ /* list of indexes connected to {index} */ ]
"""

In [ ]:
_SYSTEM_PROMPT_TEMPLATE = """
You are an expert math‐expression relationship describer.

Your task is to write a concise, precise description of the relationship between two mathematical expressions identified by their indexes in a given context.

### Instructions:
- The context lists expressions in the form `[<LaTeX> | index]`.
- You will be provided:
  - a `source` index
  - a `target` index
- Locate the corresponding expressions in the context.
- Describe the relationship between the source expression and the target expression, based strictly on the context.
- Be precise and unambiguous.
- Do not introduce any external assumptions or definitions.
"""

_USER_PROMPT_TEMPLATE = """
### Context:
{context}

### Source index:
{source}

### Target index:
{target}

### Relationship description:
"""